In [ ]:
""" Read training data files and produces data labels and statistics on policy, and sgf files: 

    # Position meta-data:
    chunk_id:          chunk number within training file
    game_id:           game number within the chunk (start at 1)
    game_len_raw:      game length, raw
    game_length:       game length, floored by 20
    move_count_raw:    move count, raw
    move_count:        move count, floored by 20
    side_to_play:      color to play (Black/White)
    played_by:         move belongs to winner/loser of the game
    picked_move        board coordinate of move picked by temperature
    
    # Policy statistics:
    p_picked_raw       policy of picked move, raw
    p_picked           policy of picked move, floored by 0.02
    picked_rank        the rank of the move picked by temperature in the search policy (best move rank = 1)
    p_pass_raw:        policy probability of pass move
    p_pass:            policy probability of pass move, floored by 0.02
    pass_rank:         the rank of the pass move in the search policy 
    p_max_raw:         policy max (=policy of best move), raw
    p_max:             policy max (=policy of best move), floored by 0.02
    p_best5_raw:       cumulated policy probability of top  5 moves, raw
    p_best5 :          cumulated policy probability of top  5 moves, floored by 0.02
    p_best10_raw:      cumulated policy probability of top 10 moves, raw
    p_best10:          cumulated policy probability of top 10 moves, floored by 0.02
    p_pass_raw:        policy probability of pass move
    p_pass:            policy probability of pass move, floored by 0.02
    norm_cost_raw      normalized computing cost of genrating the next move in current position = tree reuse ratio
    norm_cost          normalized computing cost of genrating the next move in current position, floored by 0.02
    
    # Commented sgf:
    chunk_XXXX_game_YYYY.sgf : a text file in FF4 format of the game ID YYYY of chunk ID XXXX
"""

""" USAGE
    . Download and unzip a training data file from https://leela.online-go.com/training/
    . In "__main__", configure 'main_path'. And 'training_id' with the chosen network hash (ex.: '68824bbc')
    . Select a bunch of training data chunks ('train_68824bbc_XXXX.gz' e.g.), unzip and save them
      in a directory having the same name as training file (e.g. 'C:/main_path/train_68824bbc/').
      100 chunks correspond to 3200 games and a csv table of 550k-700k positions, manipulable with Excel
    . This script assumes there is a series of unzipped training chunks (e.g. 'train_68824bbc_2500')
      in the here above mentioned directory. Also, it assumes chunks are of consecutive numbers and there is
      no other files in the directory but the chunks.
    . If first chunk is not chunk 0, or if you wish to start decoding with another chunk, indicate in "__main__"
      the 'start_chunk_id'. Script can be terminated at any time as sgf are created on the fly and stats are
      written to the csv file in append mode.
    . Games and stats table are created in a directory named as train_id with '_output' extension.
    . Line 69/70, set decimal separator.
    . For debug, set verbosity to True.
    . NB: due to comments inserted after each move, these sgf cannot be back-processed with 'dump_supervised'
      command with LZ in console mode. Need to strip comments and add result ('RE[B+Resign]' e.g.)

"""

verbosity = False

import os
from copy import deepcopy

# Global constants & utilities:
# ----------------------------

# Empty input plane (81 hexadecimal 0) 
empty_board_hex, empty_board = '0' * 91, '.' * 364

p_floor, c_floor = 0.02, 20   # Policy and move count/game length floor values (2% and 20 moves resp.)

# decimal_sep = ','             
decimal_sep = '.'       #  <=== To be customized

stat_header = 'chunk_id;game_id;game_length_raw;game_length;move_count_raw;move_count;side_to_play;played_by;picked_move;p_picked_raw; p_picked;picked_rank;p_max_raw;p_max;p_best5_raw;p_best5;p_best10_raw;p_best10;p_pass_raw;p_pass;pass_rank;norm_cost_raw;norm_cost\n'

alphanum_board = 'ABCDEFGHJKLMNOPQRST'
alphanum_sgf   = 'abcdefghijklmnopqrs'

str2black = {'0':'....', '1':'...X', '2':'..X.', '3':'..XX', '4':'.X..', '5':'.X.X', '6':'.XX.', '7':'.XXX',
             '8':'X...', '9':'X..X', 'a':'X.X.', 'b':'X.XX', 'c':'XX..', 'd':'XX.X', 'e':'XXX.', 'f':'XXXX'}
str2white = {'0':'....', '1':'...O', '2':'..O.', '3':'..OO', '4':'.O..', '5':'.O.O', '6':'.OO.', '7':'.OOO',
             '8':'O...', '9':'O..O', 'a':'O.O.', 'b':'O.OO', 'c':'OO..', 'd':'OO.O', 'e':'OOO.', 'f':'OOOO'}

# Parametric Floor function
# --------------------------
def floored(x,t):
    return int(x/t)*t

def flat2xy(move):
    """ Convert flat coordinate in [0:360] to x,y coordinate (lowest value (1,1)) """
    if move == 361: return None
    else: return move // 19 + 1, move % 19 + 1

def flat2board(move):
    """ Convert flat coordinate in [0:361] to alphanum board coordinate (lowest value A1, 361 = Pass) """
    x,y = move // 19 + 1, move % 19 + 1
    if move == 361: return 'Pass'
    else: return alphanum_board[y - 1] + str(x)

def flat2sgf(move):
    """ """
    x,y = move // 19 + 1, move % 19 + 1
    if move == 361: return 'tt'
    else: return alphanum_sgf[y - 1] + alphanum_sgf[19-x]

# Classes:
# -------

class Position:
    """ Game position corresponding to a training sample in the training file """
    
    def __init__(self, training, output_directory, stat_filename):
        self.training = training
        self.output_directory = output_directory
        self.stat_filename = stat_filename
        self.sample = deepcopy(self.training.sample)
        self.side_to_move = 'Black'
        self.played_by = 'Winner' if self.sample[379] == '1' else 'Loser'
        self.black, self.white = empty_board, empty_board
        self.game_id = 1
        self.move_count = 0
        self.stat = []
        self.sgf_header = '(;FF[4]CA[UTF-8]GM[1]SZ[19]'
        self.sgf = self.sgf_header
        self.norm_cost = 1
    
   
    def get_move(self):
        """ Compares Black string in current position (self.black) to Black string in next position
            (calculated from self.training.sample), and deduces the move picked in the current position
            (mutatis mutandis for White, depending on side to move). """

        # Collect input planes hex strings of next position
        next_black_hex = self.training.sample[0] if self.training.sample[16] == '0' else self.training.sample[8]
        next_white_hex = self.training.sample[8] if self.training.sample[16] == '0' else self.training.sample[0]
        if verbosity: 
            print('next_black_hex:\n', next_black_hex)
            print('next_white_hex:\n', next_white_hex)
        
        # Convert input planes hex strings into next position text strings for Black and White
        self.black_next, self.white_next = '', ''
        for i in range(len(next_black_hex)-1):
            self.black_next += str2black[next_black_hex[i]]
            self.white_next += str2white[next_white_hex[i]]
        i += 1
        self.black_next += 'X' if next_black_hex[i] == '1' else '.'
        self.white_next += 'O' if next_white_hex[i] == '1' else '.' 

        if verbosity: 
            print('self.black_next:\n', self.black_next)
            print('self.white_next:\n', self.white_next)
            
        # If next training sample is the first position of a nestr2black[w game:
        if self.training.got_new_game:
            if verbosity: print('Got new game')
            self.picked_flat =  None   # Picked move flat coordinates
            self.picked_xy =    None   # Picked move (x,y) coordinates, starting at (1,1)
            self.picked_board = None   # Picked move go board coordinates (e.g. G16)
            self.picked_sgf =   None   # Picked move sgf FF4 coordinates (e.g. bp)
            
        # Else (continued game):
        else:
            if verbosity: print('The game will continue')
            if self.side_to_move == 'Black':
                side_to_move_position_current = self.black
                side_to_move_position_next = self.black_next
                color = 'X'
                if verbosity: 
                    print('self.side_to_move:', color)
                    print('side_to_move_position_current:\n', side_to_move_position_current)
                    print('side_to_move_position_next:\n', side_to_move_position_next)
            else: 
                side_to_move_position_current = self.white
                side_to_move_position_next = self.white_next
                color = 'O'
                if verbosity: 
                    print('self.side_to_move:', color)
                    print('side_to_move_position_current:\n', side_to_move_position_current)
                    print('side_to_move_position_next:\n', side_to_move_position_next)
        
            # Comparing next vs current strings to deduce move picked in current position
            move_index = 0
            while move_index < 361:
                if side_to_move_position_current[move_index] == '.' \
                and side_to_move_position_next[move_index] == color: break
                move_index += 1
            
            self.picked_flat  = move_index
            self.picked_xy    = flat2xy(move_index)
            self.picked_board = flat2board(move_index)
            self.picked_sgf   = ';' + self.side_to_move[0] + '[' + flat2sgf(move_index) + ']'
            if verbosity: print('> Get move: Picked move = ', self.picked_board, '(', self.picked_flat, ')')
        
    
    def get_stat(self):
        """ Process sample to extract policy stats """
        
        self.stat.append(self.move_count)                     # Move count raw
        self.stat.append(floored(self.move_count, c_floor))   # Move count (floored)
        self.stat.append(self.side_to_move)                   # Side to play    !!!!! Not updated yet !!!!
        self.stat.append(self.played_by)                      # Played by       !!!!! Not updated yet !!!!
        
                
        # The 361+1 moves policy values (362 = pass), zipped with their flat coordinates
        policy = list(zip([float(p) for p in self.sample[17: 379]] , list(range(362)))) 

        if self.training.got_new_game:  # Append 'NA' for:
            self.stat.append('NA')                            # picked_move (board coordinate)
            self.stat.append('NA')                            # p_picked_raw
            self.stat.append('NA')                            # p_picked (floored)
            self.stat.append('NA')                            # picked_rank
            picked_rank = None
        else:
            p_picked = policy[self.picked_flat][0]
            self.norm_cost_next = 1 - p_picked                # norm_cost of next position
            self.stat.append(self.picked_board)               # picked_move (board coordinate)
            self.stat.append(p_picked)                        # p_picked_raw 
            self.stat.append(floored(p_picked, p_floor))      # p_picked (floored)
        
        policy.sort(reverse=True)
        move_rank = [item[1] for item in policy]  # Rank of flat coordinates moves in sorted policy
        
        if not self.training.got_new_game: 
            picked_rank = move_rank.index(self.picked_flat)  
            self.stat.append(picked_rank + 1)                 # picked_rank (ranks starting at 1)
                
        pass_rank = move_rank.index(361)  # Rank of pass move
        
        p_max = policy[0][0]
        self.stat.append(p_max)                               # p_max_raw             
        self.stat.append(floored(p_max, p_floor))             # p_max (floored)
        
        p_best5 = p_max + policy[1][0] + policy[2][0] + policy[3][0] + policy[4][0]
        self.stat.append(p_best5)                             # p_best5_raw
        self.stat.append(floored(p_best5, p_floor))           # p_best5 (floored)
        
        p_best10 = p_best5 + policy[5][0] + policy[6][0] + policy[7][0] + policy[8][0] + policy[9][0]
        self.stat.append(p_best10)                            # p_best10_raw
        self.stat.append(floored(p_best10, p_floor))          # p_best10 (floored)
        
        p_pass = float(self.sample[378])
        self.stat.append(p_pass)                              # p_pass_raw
        self.stat.append(floored(p_pass, p_floor))            # p_pass (floored)
        self.stat.append(pass_rank + 1)                       # pass_rank

        self.stat.append(self.norm_cost)                      # norm_cost_raw
        self.stat.append(floored(self.norm_cost, p_floor))    # norm_cost (floored)
        
        # Generating sgf comment with (top10 + Pass + Picked) moves flat coordinates & policy:
        best10_moves = [' {:>3}'.format(i+1) + '  {:4}'.format(flat2board(policy[i][1]))\
                        + '  ({:>9}'.format('{:>6.4%})'.format(policy[i][0])) for i in range(10)]
        self.sgf_comment = 'C[Move {} - {} ({}) to play\n'.format(self.move_count, self.side_to_move, self.played_by)
        self.sgf_comment += 'Search policy:\n' + 'Rank  Move  ( visits%)\n'
        pass_in_top10, picked_in_top10 = False, False
        
        for i in range(10):
            self.sgf_comment += best10_moves[i]
            if i == picked_rank:
                self.sgf_comment += '   (Picked)'
                picked_in_top10 = True
            if i == pass_rank: pass_in_top10 = True
            self.sgf_comment += '\n'
        
        picked_comment, pass_comment = '', ''
        
        if not pass_in_top10:
            pass_comment = '\n' + ' {:>3}'.format(pass_rank + 1) + '  Pass' + '  ({:>9}'.format('{:>6.4%})'.format(p_pass))
            if picked_rank == pass_rank: pass_comment += '  Picked'
                                                                                     
        if not picked_in_top10 and not self.training.got_new_game and not picked_rank == pass_rank:
            picked_comment = '\n' + ' {:>3}'.format(picked_rank + 1) + '  {:4}'.format(self.picked_board) \
                                + '  ({:>9}'.format('{:>6.4%})'.format(p_picked)) + '  Picked'
        
        if not pass_in_top10:
            if picked_rank != None:
                if picked_rank > pass_rank:
                    self.sgf_comment += pass_comment + picked_comment
                else:
                    self.sgf_comment += picked_comment + pass_comment
            else:
                self.sgf_comment += pass_comment
        else:
            self.sgf_comment += picked_comment
        
        self.sgf_comment += ']'

        
    def update(self):
        """ Update position based on current position policy data and data on next move """
        
        self.sample = deepcopy(self.training.sample)
                                     
        if self.training.got_new_game:
            if self.training.got_new_chunk and not self.training.eof:
                self.current_chunk_id = self.training.chunk_id - 1
            else:
                self.current_chunk_id = self.training.chunk_id
            self.sgf += ')'
            self.save_sgf()
            self.save_stats()
            self.stat = []
            self.move_count = 0
            self.side_to_move = 'Black'
            self.played_by = 'Winner' if self.sample[379] == '1' else 'Loser'
            self.black, self.white = empty_board, empty_board 
            
            if not self.training.got_new_chunk:
                self.game_id += 1  # Incrementing game number
            else:
                self.game_id = 1   # Or resetting it to 1 if new chunk         
            
            self.sgf = self.sgf_header
            self.norm_cost = 1
            
        
        else:
            self.move_count += 1
            self.side_to_move = 'Black' if self.sample[16]  == '0' else 'White'
            self.played_by =   'Winner' if self.sample[379] == '1' else 'Loser'
            self.black, self.white = self.black_next, self.white_next
            self.norm_cost = self.norm_cost_next
            self.sgf += self.sgf_comment + self.picked_sgf
        

    def show(self):
        """ Raw print of the position, before the update """

        board=''
        for i in range(361):
            if   self.black[i]=='X': board+='X'
            elif self.white[i]=='O': board+='O'
            elif self.picked_flat == i: board+='*'
            else:
                board+='+' if (i//19-3)%6 == 0 and (i%19-3)%6 == 0 else '.'
        print('\n---------------------------------------------------')
        print('\nGame {:5d}  -  Move count:{:3d}'.format(self.game_id, self.move_count))
        print('Black (X) to move' if self.side_to_move == 'Black' else 'White (O) to move')
#         if self.picked_flat != None:
#             print('Picked move(*) prob.: {:4.2f} %'.format(self.stat[0]*100) if self.picked_flat != None else '')
        print('   A B C D E F G H J K L M N O P Q R S T')
        for i in range(18,-1,-1):
            print('%2i' %(i+1),end='')
            for j in range(19):
                print('',board[i*19+j],end='')
            print(' %2i' %(i+1))
        print('   A B C D E F G H J K L M N O P Q R S T', end = '')
        print('     Pass:(*)' if self.picked_flat == 361 else '', end = '')
        print('     Game end' if self.picked_flat == None else '')
#         print('Picked move:', self.picked_board, self.picked_sgf, self.picked_xy, self.picked_flat)
        print('\n', self.sgf_comment)


    def save_sgf(self):
        """ Save sgf file of finished game """
        
        sgf_filename = self.output_directory + 'chunk_' + str(self.current_chunk_id) + '_game_' + str(self.game_id) + '.sgf'
        with open(sgf_filename, 'w') as f:
            f.write(self.sgf)

            
    def save_stats(self):
        """ Save stats for the last game """
        
        game_stat_line = ''
        game_stat_prefix = str(self.current_chunk_id) + ';' + str(self.game_id) + ';'
        game_stat_prefix += str(self.move_count) + ';' + str(floored(self.move_count, c_floor))  # Game length
        
        for i in range(self.move_count):
            game_stat_line += game_stat_prefix
            for j in range(19):
                game_stat_line += ';' + str(self.stat[19 * i + j]).replace(".", decimal_sep)
            game_stat_line += '\n'
        
#         print('Appending game statistics to ',stat_filename,' ......')
        with open(self.stat_filename,'a') as f:
            f.write(game_stat_line)
        
    
class Training:
    """ A chunk correspond to a training file chunk (must be manually unzipped in place before use of this script).
        With get_sample() method, a Training() object returns a chunk_id and a list of 380 items corresponding
        to a training sample, while iterating over all available training file chunks in the training file directory.
        By default, iterates from chunk_id = 0. Start_id can be provided as an argument. """

    
    def __init__(self, main_path, train_id, chunk_id = 0):
        self.chunk_id = chunk_id     # By default, the first chunk of the training file
        training_name = 'train_' + train_id
        training_path = main_path + training_name + '/'
        self.file_name_prefix = training_path + training_name + '_'
        path, dirs, files = next(os.walk(training_path))
        self.last_chunk_id = chunk_id + len(files) - 1
        print('> Preparing to process', len(files),' chunks: chunk', chunk_id, ' to chunk', self.last_chunk_id)
        self.read_chunk()             # Initializes self.chunk and self.chunk_length
        self.eof = False              # End Of File condition
        self.got_new_chunk = False    # Flag to indicate that read_chunk() made the crawler jump to a new chunk
        
        
    
    def read_chunk(self):
        """ Read the chunk file corresponding to the chunk_id."""
        
        self.file_name = self.file_name_prefix + str(self.chunk_id)
        print('> Loading training data from ', self.file_name)
        with open(self.file_name) as f:
            self.chunk = f.read().split()
        self.chunk_length = int(len(self.chunk)/380)
        self.sample_id = -1  # Ready to start at first sample of the new chunk, after increment by 1
        print('> Chunk ', self.chunk_id, 'has ', self.chunk_length, 'samples')

        
    def get_sample(self):
        """ Update self.sample with the 380 lines of chunk corresponding to the next training sample """
        
        if self.sample_id == self.chunk_length - 1:      # Current sample is last sample of current chunk
            print('> Training sample {:5d}. End of chunk {:4d} reached'.format(self.sample_id, self.chunk_id))
            if self.chunk_id == self.last_chunk_id:    # Current chunk is last chunk of training data
                print('> Chunk {:3d} was the last chunk.'.format(self.chunk_id))
                self.eof = True
                self.got_new_chunk = True    # Fake 'new chunk' to prompt stats & sgf saving before stopping
                self.got_new_game  = True    # Fake 'new game' to  prompt stats & sgf saving before stopping
                return
            else:
                self.chunk_id += 1
                self.read_chunk()
                self.got_new_chunk = True
        else:
            self.got_new_chunk = False
        
        self.sample_id += 1                      # Increment sample index
        start = self.sample_id * 380
        self.sample = self.chunk[start : start + 380]  # Updating sample.

        # Check if sample matches an empty board position
        self.got_new_game = self.sample[0] == empty_board_hex and self.sample[8] == empty_board_hex  
    
    
    def raw_print(self):
        """ Raw print of the training sample """
        
        print('\nChunk_id:', self.chunk_id, ' - Sample_id:', self.sample_id)
        print('\nInput planes:')
        for i in range(16):
            print('{:3d}'.format(i), training.sample[i])
            if i == 7: print('')
        print('Side to move:')
        print('{:3d}'.format(16), training.sample[16])
        print('Policy:')
        for i in range(72):
            for j in range (17 + 5 * i, 22 + 5 * i):
                print('{:3d}: {:8.7f}'.format(j - 17, float(training.sample[j])), ';', end = '')
            print('')
        print('{:3d}: {:8.7f}'.format(360, float(training.sample[377])), ';', end = '')
        print('{:3d}: {:8.7f}'.format(361, float(training.sample[378])))
        print('Played by:')
        print('{:3d}: '.format(379), training.sample[379])
        

if __name__ == "__main__":
    
    # Selecting training data file:
    # ----------------------------
    # train_id = '62b5417b'   # Training hash ELF v0
    # train_id = 'd13c4099'   # Training hash ELF v1
    # train_id = '2da87ea8'   # Training hash from LZ184 - 26-Oct-2018
    train_id = 'f50dc27c'   #  Training hash from LZ186                   #  <=== To be customized
    
    
    training_name = 'train_' + train_id

    # Creating output directory for stats and sgf:
    # -------------------------------------------    
    main_path = 'C:/Users/Username/Downloads/'                            #  <=== To be customized
    output_directory = main_path + training_name + '_output/'
    stat_filename = output_directory + training_name + '_stats.csv'
    directory = os.path.dirname(stat_filename)
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)
    
    print('output_directory:', output_directory)
    
    with open(stat_filename,'w') as f:
        f.write(stat_header)
    
    
    # Looping over training data:
    # --------------------------

    training = Training(main_path, train_id, chunk_id = 0)
    training.get_sample()
    if verbosity: training.raw_print()
    position = Position(training, output_directory, stat_filename)
    
    while not training.eof:        
        training.get_sample()
        if verbosity: training.raw_print()
        position.get_move()
        position.get_stat()
        if verbosity: position.show()
        position.update()
        

 